In [14]:
import torch 
import numpy as np
import random 


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

random.seed — to ensure that all random calls are repeatable
np.random.seed — NumPy repeatability
torch.manual_seed — repeatability of tensors and initializations on the CPU
torch.cuda.manual_seed_all — repeatability on the GPU (if there any)

In [ ]:
set_seed(42)
print(torch.rand(3))


tensor([0.8823, 0.9150, 0.3829])


In [ ]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader 

train_ds = MNIST(
    root="./data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_ds = MNIST(
    root="./data",
    train=False,
    download=True,
    transform=ToTensor()
)

print(len(train_ds))
print(len(test_ds))

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=1024, shuffle=False)





60000
10000
